<a href="https://colab.research.google.com/github/IVMRanepa/courses.shift01/blob/main/Technical_security_measures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**📎 Пример кода RBAC (Python):**

In [ ]:
roles={"admin":["read","write"],"user":["read"]}
users={"alice":"admin","bob":"user"}
print("alice can write:", "write" in roles[users["alice"]])

alice can write: True


**Пример шифрования (Python, Fernet):**

In [ ]:
from cryptography.fernet import Fernet
key=Fernet.generate_key()
f=Fernet(key)
cipher=f.encrypt(b"Secret")
print("Decrypted:", f.decrypt(cipher))

Decrypted: b'Secret'


**Задание 1 — Симуляция контроля доступа (RBAC)**

In [ ]:
# task01_rbac.py
roles = {
    "admin": ["read", "write", "delete"],
    "user": ["read"],
}
users = {"alice": "admin", "bob": "user"}

def can(user, action):
    role = users.get(user)
    return action in roles.get(role, [])

print("alice can delete:", can("alice","delete"))
print("bob can delete:", can("bob","delete"))

alice can delete: True
bob can delete: False


✅ Вывод: RBAC уменьшает риски эскалации прав.
📌 Рекомендация: применять принцип минимальных привилегий.
🔗 GitHub: [OWASP RBAC Example](https://github.com/OWASP)

**Книга Д. Г. Грэм «Этичный хакинг», Часть III — социальная инженерия / глава о фишинге и упражнениях**

Набор безопасных демонстрационных заданий на Python 3 для запуска в Google Colab. Они нацелены на обучение распознаванию и противодействию мошенничеству, а не на создание вредоносных инструментов; все задания сопровождаются решениями, подробными комментариями, выводами и инструкцией по размещению в GitHub.

(В книге есть подробное обсуждение фишинга, дипфейков и упражнений по массовому фишингу и аудиту SMTP — см. главы и упражнения).

Ниже — план и готовые компактные блоки кода (готовые для вставки в Colab). Каждое задание: цель → код → комментарии → ожидаемый вывод → вывод/учебные моменты. В конце — структура GitHub-репозитория и рекомендации по этике/безопасности.

1) **Задание A** — «Анализ заголовков письма (email header analyzer)»

**Цель:** научить/показать, какие признаки подделки письма искать в заголовках (From vs Return-Path, Received-цепочка, наличие DKIM/SPF записей). (Базовая проверка — локальная строковая и синтаксическая; опционально — DNS-проверка SPF/DKIM в Colab, если разрешён доступ в сеть.)

In [ ]:
!pip install dnspython
!pip install python3-resolver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 9.0 MB/s eta 0:00:00


In [ ]:
# Colab cell: Email header analyzer (Python 3)
from email import policy
from email.parser import Parser
import re
import dns.resolver   # pip install dnspython (Colab may need internet)

# Функция для парсинга заголовков email
def parse_headers(raw_email: str):
    # Используем Parser для извлечения заголовков из сырого текста письма
    msg = Parser(policy=policy.default).parsestr(raw_email)
    # Преобразуем заголовки в словарь для удобства доступа
    headers = dict(msg.items())
    return headers, msg

# Функция для выполнения проверок на подозрительные признаки в заголовках
def suspicious_checks(headers):
    findings = []
    # 1. Сравнение From и Return-Path
    from_addr = headers.get('From', '')
    return_path = headers.get('Return-Path', '')
    # Если Return-Path существует и не совпадает с From, это подозрительно
    if return_path and (return_path not in from_addr):
        findings.append("Mismatch: Return-Path и From отличаются.")
    # 2. Проверка наличия DKIM-Signature
    # Отсутствие DKIM-подписи может указывать на подделку
    if 'DKIM-Signature' not in headers:
        findings.append("DKIM-Signature отсутствует.")
    # 3. Анализ цепочки Received
    # Получаем все поля Received (может быть несколько)
    received = headers.get_all('Received') if hasattr(headers, 'get_all') else headers.get('Received')
    if received:
        # Объединяем все записи Received в одну строку
        rcv_text = '\n'.join(received) if isinstance(received, list) else str(received)
        # Ищем IP-адреса в цепочке Received
        ips = re.findall(r'\[?(\d{1,3}(?:\.\d{1,3}){3})\]?', rcv_text)
        if ips:
            findings.append(f"IP-адреса в Received: {ips}")
        # Проверяем длину цепочки Received (много 'by ' указывает на ретрансляцию)
        if rcv_text.count('by ') > 5:
            findings.append("Длинная цепочка Received (возможно ретрансляция через копию почтовых серверов).")
    return findings

# Пример использования
raw = """Return-Path: <spoof@example.com>
From: "Bank Support" <support@bank.example>
Received: from unknown (HELO mail.example) (192.0.2.45)
DKIM-Signature: v=1; a=rsa-sha256; ..."""
headers, msg = parse_headers(raw)
print("Headers keys:", list(headers.keys()))
print("Findings:", suspicious_checks(headers))

Headers keys: ['Return-Path', 'From', 'Received', 'DKIM-Signature']
Findings: ['Mismatch: Return-Path и From отличаются.', "IP-адреса в Received: ['192.0.2.45']"]


**Комментарии:**

Этот ноутбук разбирает заголовки и выделяет простые аномалии.

Для реальной проверки SPF/DKIM нужно запросить DNS (TXT) и/или валидировать подпись DKIM — код может включать dns.resolver.resolve(domain, 'TXT') и библиотеку dkim для верификации. В Colab такие запросы возможны, если разрешён интернет.

**Ожидаемый вывод:** список подозрительных находок (Mismatch Return-Path, отсутствие DKIM, IP-адреса в Received и т.п.).

**Вывод/учебные моменты:** анализ заголовков — первый и безопасный уровень защиты; сотрудника учили бы смотреть на Return-Path, Received-цепочку и наличие DKIM/SPF/DMARC. (См. обсуждение подделки писем в книге).

[Библиотека программиста] Дэние…

In [ ]:
# Colab cell: Email header analyzer (Python 3)
from email import policy
from email.parser import Parser
import re
import dns.resolver   # pip install dnspython (Colab may need internet)

# Функция для парсинга заголовков email
def parse_headers(raw_email: str):
    # Используем Parser для извлечения заголовков из сырого текста письма
    msg = Parser(policy=policy.default).parsestr(raw_email)
    # Преобразуем заголовки в словарь для удобства доступа
    headers = dict(msg.items())
    return headers, msg

# Функция для выполнения проверок на подозрительные признаки в заголовках
def suspicious_checks(headers):
    findings = []
    # 1. Сравнение From и Return-Path
    from_addr = headers.get('From', '')
    return_path = headers.get('Return-Path', '')
    # Если Return-Path существует и не совпадает с From, это подозрительно
    if return_path and (return_path not in from_addr):
        findings.append("Mismatch: Return-Path и From отличаются.")
    # 2. Проверка наличия DKIM-Signature
    # Отсутствие DKIM-подписи может указывать на подделку
    if 'DKIM-Signature' not in headers:
        findings.append("DKIM-Signature отсутствует.")
    # 3. Анализ цепочки Received
    # Получаем все поля Received (может быть несколько)
    received = headers.get_all('Received') if hasattr(headers, 'get_all') else headers.get('Received')
    if received:
        # Объединяем все записи Received в одну строку
        rcv_text = '\n'.join(received) if isinstance(received, list) else str(received)
        # Ищем IP-адреса в цепочке Received
        ips = re.findall(r'\[?(\d{1,3}(?:\.\d{1,3}){3})\]?', rcv_text)
        if ips:
            findings.append(f"IP-адреса в Received: {ips}")
        # Проверяем длину цепочки Received (много 'by ' указывает на ретрансляцию)
        if rcv_text.count('by ') > 5:
            findings.append("Длинная цепочка Received (возможно ретрансляция через копию почтовых серверов).")
    return findings

# Пример использования
raw = """Return-Path: <spoof@example.com>
From: "Bank Support" <support@bank.example>
Received: from unknown (HELO mail.example) (192.0.2.45)
DKIM-Signature: v=1; a=rsa-sha256; ..."""
headers, msg = parse_headers(raw)
print("Headers keys:", list(headers.keys()))
print("Findings:", suspicious_checks(headers))

Headers keys: ['Return-Path', 'From', 'Received', 'DKIM-Signature']
Findings: ['Mismatch: Return-Path и From отличаются.', "IP-адреса в Received: ['192.0.2.45']"]


2) **Задание B** — «Фичи для распознавания фишинговых URL + простая модель»

**Цель:** научить определять подозрительные URL по набору простых признаков; показать, как собрать простую модель ML (логистическая регрессия) на синтетических данных — полезно для учебных целей (в Colab). Важно: не использовать реальных уязвимых списков; датасет — синтетический/демонстрационный.

In [ ]:
!pip install tldextract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 1.9 MB/s eta 0:00:00


In [ ]:
# Colab cell: URL phishing heuristic + tiny ML demo
import pandas as pd
import tldextract
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def url_features(url):
    u = urlparse(url)
    ext = tldextract.extract(url)
    host = u.netloc
    path = u.path
    feats = {}
    feats['len_url'] = len(url)
    feats['has_ip_host'] = bool(re.match(r'^\d+\.\d+\.\d+\.\d+$', ext.domain))
    feats['count_subdomains'] = host.count('.') - (1 if ext.subdomain=='' else 0)
    feats['has_at'] = '@' in url
    feats['len_path'] = len(path)
    feats['punycode'] = 'xn--' in url
    return feats

# Create tiny synthetic dataset
good = [
 "https://www.example.com/login",
 "https://bank.example.com/account",
 "https://github.com/user/repo"
]
bad = [
 "http://192.0.2.123/login", # IP host
 "http://secure-example.com.user-info.tk/login", # many subdomains
 "http://paypal.verify.xn--pypal-123.com/confirm" # punycode-like
]
rows=[]
for u in good:
    f=url_features(u); f['label']=0; f['url']=u; rows.append(f)
for u in bad:
    f=url_features(u); f['label']=1; f['url']=u; rows.append(f)
df=pd.DataFrame(rows).sample(frac=1, random_state=1)
X=df[['len_url','has_ip_host','count_subdomains','has_at','len_path','punycode']].astype(float)
y=df['label']
clf=LogisticRegression().fit(X,y)
print(classification_report(y, clf.predict(X)))
df[['url','label']].assign(pred=clf.predict(X))

              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       1.00      0.67      0.80         3

    accuracy                           0.83         6
   macro avg       0.88      0.83      0.83         6
weighted avg       0.88      0.83      0.83         6



,url,label,pred
2,https://github.com/user/repo,0,0
1,https://bank.example.com/account,0,0
4,http://secure-example.com.user-info.tk/login,1,1
0,https://www.example.com/login,0,0
3,http://192.0.2.123/login,1,0
5,http://paypal.verify.xn--pypal-123.com/confirm,1,1


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['label'].plot(kind='hist', bins=20, title='label')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['pred'].plot(kind='hist', bins=20, title='pred')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_9['url'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_9, x='index', y='url', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_10['url'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_10, x='label', y='url', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_11['url'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_11, x='pred', y='url', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

**Комментарии:**

**Модель и dataset — учебные**, слишком малы для продакшна; цель — показать pipeline: извлечение признаков → обучение → интерпретация.

В реальной системе добавляют фичи: age of domain, регистрационные данные WHOIS, наличие шортенеров, рейтинг репутации, ML на токенах URL и др.

**Вывод/учебные моменты**: простые эвристики уже ловят многие трюки; ML требует аккуратных, этичных данных. (Связано с масштабным фишингом в упражнениях книги — там описываются векторные подходы, но книга также подчёркивает этику и законность).

3) **Задание C** — «Интерактивный тренажёр: распознай фишинговое письмо» (IPython widgets)

**Цель:** интерактивный тест для сотрудников — показать типичные приёмы социальной инженерии и тренировать внимательность (объяснения по каждой ошибке).

In [ ]:
# Colab cell: simple interactive quiz (works in Colab)
from IPython.display import display, Markdown
import ipywidgets as widgets

email_example = {
    'subject': 'Срочно: подтвердите платеж',
    'from': 'support@fake-bank.example',
    'body': 'Здравствуйте, ваш платёж не прошёл. Нажмите https://bank.verify-secure.example/login и войдите.'
}

display(Markdown("### Пример письма"))
display(Markdown(f"**From:** {email_example['from']}"))
display(Markdown(f"**Subject:** {email_example['subject']}"))
display(Markdown(email_example['body']))

choices = ['Подозрительный URL', 'Необходимо срочно', 'Официальный адрес', 'Нормально']
sel = widgets.SelectMultiple(options=choices)
btn = widgets.Button(description='Проверить')

def on_click(b):
    picked = set(sel.value)
    feedback=[]
    if 'Подозрительный URL' in picked:
        feedback.append("- Верно: URL содержит подозрительный домен (verify-secure.example).")
    else:
        feedback.append("- Пропущен подозрительный URL.")
    if 'Необходимо срочно' in picked:
        feedback.append("- Верно: чувство срочности — классический приём социальной инженерии.")
    display(Markdown("\n".join(feedback)))

btn.on_click(on_click)
display(sel, btn)

### Пример письма

**From:** support@fake-bank.example

**Subject:** Срочно: подтвердите платеж

Здравствуйте, ваш платёж не прошёл. Нажмите https://bank.verify-secure.example/login и войдите.

SelectMultiple(options=('Подозрительный URL', 'Необходимо срочно', 'Официальный адрес', 'Нормально'), value=()…

Button(description='Проверить', style=ButtonStyle())

- Пропущен подозрительный URL.

**Комментарии:** интерактивный квиз прост и может быть расширен до набора писем, баллов и отчёта для тренера.

**Вывод:** практика распознавания — эффективнее чтения теории; можно внедрять такие тренажёры в курсы по безопасности (книга рекомендует упражнения и лаборатории по социальному инжинирингу).

4) **Задание D** — «Проверка надёжности пароля с учётом персональной информации (защита от социальной инженерии)»

**Цель:** показать, почему пароли, содержащие персональные данные (имя, день рождения, организация), легко подбираются мошенниками; предложить рекомендации.

In [ ]:
# Colab cell: password strength + personal info check
import re

def password_checks(pwd, personal_tokens):
    issues=[]
    if len(pwd) < 8: issues.append("Слишком короткий (<8).")
    if not re.search(r'[A-Z]', pwd): issues.append("Нет заглавной буквы.")
    if not re.search(r'[0-9]', pwd): issues.append("Нет цифр.")
    # personal info check
    low = pwd.lower()
    for t in personal_tokens:
        if t.lower() in low:
            issues.append(f"Содержит персональную информацию: {t}")
    return issues

# Example
personal = ["ivan", "ivanov", "1986", "companyname"]
print(password_checks("Ivan1986!", personal))

['Содержит персональную информацию: ivan', 'Содержит персональную информацию: 1986']


**Комментарии:** демонстрация того, как внутренняя разведка (OSINT) помогает злоумышленникам — поэтому политика паролей и MFA критичны. (В книге есть разделы про сбор информации и её использование).

**Этические и юридические замечания (обязательно)**

Все представленные задания защитные/учебные и не содержат» инструкций по созданию фишинговых страниц, спуфинга SMTP, клонирования голосов и т. п. (эти темы обсуждаются в книге как угроза и как лабораторные упражнения в контролируемой среде). Если вы хотите разобрать более «практические» упражнения (например, эмулировать фишинг для корпоративных тренингов), это должно выполняться только в пределах согласованной учебной среды с письменным согласием участников и под руководством компетентного инструктора/юриста.

**Задание 5.** Анализ заголовков писем (синтетические данные)

In [ ]:
# Генерация синтетических заголовков
synthetic_headers = [
    """Return-Path: <spoof@fakebank.com>
From: "Bank Support" <support@bank.com>
Received: from host.fake (192.0.2.10)
""",
    """Return-Path: <info@company.org>
From: "Company Info" <info@company.org>
Received: from relay.company.org (203.0.113.5)
DKIM-Signature: v=1; a=rsa-sha256; ...
"""
]

def analyze_header(raw):
    issues=[]
    if "DKIM-Signature" not in raw:
        issues.append("Нет DKIM-подписи")
    if "spoof@" in raw:
        issues.append("Return-Path подозрительный")
    if "192.0.2." in raw:
        issues.append("Признак тестового IP (RFC 5737)")
    return issues

for i, h in enumerate(synthetic_headers, 1):
    print(f"\n=== Пример {i} ===")
    print(analyze_header(h))


=== Пример 1 ===
['Нет DKIM-подписи', 'Return-Path подозрительный', 'Признак тестового IP (RFC 5737)']

=== Пример 2 ===
[]


**Задание 6.** Распознавание подозрительных URL (синтетический датасет)

In [ ]:
import pandas as pd
import random, string

def random_domain():
    return ''.join(random.choice(string.ascii_lowercase) for _ in range(6)) + ".com"

good_urls = [f"https://{random_domain()}/login" for _ in range(5)]
bad_urls = [
    f"http://192.0.2.{i}/secure" for i in range(5,10)
] + [
    f"http://verify.{random_domain()}.info/login" for _ in range(5)
]

data = pd.DataFrame({
    "url": good_urls + bad_urls,
    "label": [0]*len(good_urls) + [1]*len(bad_urls)
})

print(data.head())

                        url  label
0  https://hywalr.com/login      0
1  https://rsdwxk.com/login      0
2  https://omcvdh.com/login      0
3  https://vcbnls.com/login      0
4  https://obdbhf.com/login      0


👉 **Учебный момент:** мы создаём «хорошие» и «плохие» URL случайно, чтобы потренироваться на классификации.

**Задание 7.** Тренажёр «распознай письмо» (синтетические письма)

In [ ]:
from IPython.display import display, Markdown
import ipywidgets as widgets
import random

synthetic_emails = [
    {
        "subject": "Срочно: подтвердите платёж",
        "from": "support@fakebank.com",
        "body": "Нажмите http://secure.fakebank.com/login для подтверждения."
    },
    {
        "subject": "Уведомление: новая рассылка",
        "from": "info@newsletter.org",
        "body": "Спасибо за подписку! Никаких действий не требуется."
    }
]

email = random.choice(synthetic_emails)
display(Markdown(f"**From:** {email['from']}"))
display(Markdown(f"**Subject:** {email['subject']}"))
display(Markdown(email['body']))

options = ['Подозрительно', 'Нормально']
sel = widgets.RadioButtons(options=options)
btn = widgets.Button(description="Проверить")

def on_click(b):
    if "fakebank" in email['from']:
        correct = 'Подозрительно'
    else:
        correct = 'Нормально'
    result = "✅ Верно" if sel.value == correct else f"❌ Ошибка. Правильный ответ: {correct}"
    display(Markdown(result))

btn.on_click(on_click)
display(sel, btn)

**From:** support@fakebank.com

**Subject:** Срочно: подтвердите платёж

Нажмите http://secure.fakebank.com/login для подтверждения.

RadioButtons(options=('Подозрительно', 'Нормально'), value='Подозрительно')

Button(description='Проверить', style=ButtonStyle())

✅ Верно

**Задание 8.** Проверка паролей (синтетические пользователи)

In [ ]:
import random

# синтетические "персональные токены"
tokens = ["alex", "petrov", "2000", "corp"]

passwords = ["Alex2000!", "Qwerty123", "SecurePass!45"]

def check_password(pwd, tokens):
    issues=[]
    if len(pwd)<8: issues.append("короткий")
    if any(t in pwd.lower() for t in tokens): issues.append("содержит персональные данные")
    return issues or ["надёжный"]

for p in passwords:
    print(p, "->", check_password(p, tokens))

Alex2000! -> ['содержит персональные данные']
Qwerty123 -> ['надёжный']
SecurePass!45 -> ['надёжный']


👉 **Учебный момент:** пароли формально разные, но некоторые содержат персональные данные.

📌 **Выводы**

Все данные полностью синтетические → безопасно для учебных ноутбуков.

Показываем принципы противодействия социальной инженерии: анализ заголовков, проверка URL, внимательность к письмам, пароли.

Задания легко запускать в Google Colab.

**Задание 9.** Анализ телефонных звонков (voice phishing, «вишинг»)

Идея: сгенерируем записи звонков в виде текстовых расшифровок и проверим их на наличие «триггеров» — срочность, угрозы, просьба назвать данные.

In [ ]:
calls = [
    "Здравствуйте! Это служба безопасности банка. Срочно назовите код из SMS, иначе счёт будет заблокирован.",
    "Добрый день! Это техподдержка компании. Мы обновляем систему, никаких действий от вас не требуется.",
]

triggers = ["срочно", "код", "заблокирован", "пароль"]

def analyze_call(text):
    text_l = text.lower()
    return [t for t in triggers if t in text_l]

for c in calls:
    print(f"\nЗвонок: {c}")
    print("Триггеры:", analyze_call(c))


Звонок: Здравствуйте! Это служба безопасности банка. Срочно назовите код из SMS, иначе счёт будет заблокирован.
Триггеры: ['срочно', 'код', 'заблокирован']

Звонок: Добрый день! Это техподдержка компании. Мы обновляем систему, никаких действий от вас не требуется.
Триггеры: []


👉 **Учебный момент:** сотрудники учатся слышать «сигнальные слова» в мошеннических звонках.

**Задание 10.** Генерация фишинговых SMS («смишинг»)

**Идея:** случайно сгенерировать SMS-сообщения и классифицировать их как «подозрительные» или «нормальные».

In [ ]:
import random

normal_sms = [
    "Ваш заказ №123 готов к выдаче.",
    "Напоминаем: завтра встреча в 14:00.",
]

phish_sms_templates = [
    "Срочно! Ваш аккаунт заблокирован. Перейдите по ссылке {url}",
    "Вы выиграли приз! Подтвердите данные здесь: {url}"
]

urls = ["http://fake-update.com", "http://secure-verify.org"]

sms_dataset = normal_sms + [t.format(url=random.choice(urls)) for t in phish_sms_templates]

for s in sms_dataset:
    print(s, "->", "Подозрительное" if "http://" in s else "Нормальное")

Ваш заказ №123 готов к выдаче. -> Нормальное
Напоминаем: завтра встреча в 14:00. -> Нормальное
Срочно! Ваш аккаунт заблокирован. Перейдите по ссылке http://fake-update.com -> Подозрительное
Вы выиграли приз! Подтвердите данные здесь: http://secure-verify.org -> Подозрительное


**Задание 11.** Социальные сети: подозрительные сообщения

**Идея:** смоделируем личные сообщения в соцсетях (например, «поделитесь фото паспорта», «установите приложение») и проверим на красные флаги.

In [ ]:
messages = [
    "Привет! Можешь прислать фото паспорта для конкурса?",
    "Добрый день, заходите завтра на лекцию в 10 утра.",
    "Скачайте новое приложение по ссылке: http://dangerous-app.net"
]

flags = ["паспорт", "ссылка", "скачайте", "приложение"]

def check_message(msg):
    found = [f for f in flags if f in msg.lower()]
    return found or ["безопасно"]

for m in messages:
    print(m, "->", check_message(m))

Привет! Можешь прислать фото паспорта для конкурса? -> ['паспорт']
Добрый день, заходите завтра на лекцию в 10 утра. -> ['безопасно']
Скачайте новое приложение по ссылке: http://dangerous-app.net -> ['скачайте', 'приложение']


👉 **Учебный момент:** люди должны насторожиться при просьбах выслать документы или перейти по ссылке.

**Задание 12.** Генерация фальшивых сайтов (детектор шаблонов)

**Идея:** сгенерировать «HTML страниц» (строкой), и по ключевым признакам показать, какие элементы часто встречаются во фишинговых сайтах.

In [ ]:
pages = [
    "<html><body><h1>Добро пожаловать в Банк</h1><form action='http://fake.com/login'><input name='password'></form></body></html>",
    "<html><body><h1>Наш университет</h1><p>Новости и расписание</p></body></html>"
]

def analyze_page(html):
    issues=[]
    if "http://" in html and "form" in html:
        issues.append("Форма на небезопасном (http) сайте")
    if "password" in html:
        issues.append("Запрос пароля на подозрительном сайте")
    return issues or ["безопасно"]

for p in pages:
    print("Страница:", analyze_page(p))

Страница: ['Форма на небезопасном (http) сайте', 'Запрос пароля на подозрительном сайте']
Страница: ['безопасно']


👉 **Учебный момент:** даже упрощённый анализ показывает, почему нужно обращать внимание на HTTPS и контекст формы.

**Задание 13.** Поведенческая психология (симуляция)

**Идея:** показать, как мошенники используют «социальные триггеры» (авторитет, срочность, выгода).

In [ ]:
scenarios = [
    ("Это директор. Срочно отправьте мне отчёт.", ["авторитет", "срочность"]),
    ("Вы получили премию. Чтобы забрать, перейдите по ссылке.", ["выгода", "ссылка"]),
    ("Коллега просит перенести встречу.", [])
]

for text, expected in scenarios:
    print(f"\nСценарий: {text}")
    print("Триггеры:", expected if expected else "Нет")


Сценарий: Это директор. Срочно отправьте мне отчёт.
Триггеры: ['авторитет', 'срочность']

Сценарий: Вы получили премию. Чтобы забрать, перейдите по ссылке.
Триггеры: ['выгода', 'ссылка']

Сценарий: Коллега просит перенести встречу.
Триггеры: Нет


👉 **Учебный момент:** распознавание приёмов социальной инженерии.

**Задание 14.** Лог-файлы входа (симуляция брутфорса)

**Идея:** сгенерировать логи входа и искать аномалии — много попыток подряд с разных IP.

In [ ]:
import random

logs = [
    {"user": "ivan", "ip": "198.51.100." + str(random.randint(1,5)), "status": "FAIL"}
    for _ in range(5)
] + [
    {"user": "ivan", "ip": "198.51.100.3", "status": "SUCCESS"}
]

for l in logs:
    print(l)

fails = sum(1 for l in logs if l["status"]=="FAIL")
print("\nКоличество неудачных попыток:", fails)
print("Подозрительно" if fails>3 else "Норма")

{'user': 'ivan', 'ip': '198.51.100.2', 'status': 'FAIL'}
{'user': 'ivan', 'ip': '198.51.100.1', 'status': 'FAIL'}
{'user': 'ivan', 'ip': '198.51.100.3', 'status': 'FAIL'}
{'user': 'ivan', 'ip': '198.51.100.2', 'status': 'FAIL'}
{'user': 'ivan', 'ip': '198.51.100.1', 'status': 'FAIL'}
{'user': 'ivan', 'ip': '198.51.100.3', 'status': 'SUCCESS'}

Количество неудачных попыток: 5
Подозрительно


👉 **Учебный момент:** если много «FAIL» перед успешным входом → возможно подбор пароля.

**Задание 15.** Поддельные новости («фейк-ньюс»)

**Идея:** сгенерировать тексты новостей и искать эмоциональную окраску или «сенсационные» слова.

In [ ]:
texts = [
    "СРОЧНО! Ваша компания разорилась. Кликайте по ссылке, чтобы узнать подробности.",
    "Сегодня солнечно, температура +20°.",
]

flags = ["срочно", "кликайте", "шок", "сенсация"]

for t in texts:
    print(t, "->", any(f in t.lower() for f in flags))

СРОЧНО! Ваша компания разорилась. Кликайте по ссылке, чтобы узнать подробности. -> True
Сегодня солнечно, температура +20°. -> False


👉 **Учебный момент:** «clickbait» — один из способов социальной инженерии.

**Задание 16.** Ролевая игра: «директорская атака» (CEO fraud)

**Идея:** сгенерировать письма от «директора» с просьбой перевести деньги.

In [ ]:
letters = [
    "Это директор. Срочно переведите 5000 руб. на счёт 123-456.",
    "Добрый день, завтра собрание в 10:00.",
]

for l in letters:
    if "срочно" in l.lower() and "счёт" in l.lower():
        print(l, "-> Подозрительно (финансовое требование + срочность)")
    else:
        print(l, "-> Норма")

Это директор. Срочно переведите 5000 руб. на счёт 123-456. -> Подозрительно (финансовое требование + срочность)
Добрый день, завтра собрание в 10:00. -> Норма


👉 **Учебный момент:** «спуфинг директора» очень распространён.

**Задание 17.** Поддельные анкеты («опросы»)

**Идея:** мошенники любят рассылать «опросы» с просьбой ввести личные данные.

In [ ]:
forms = [
    {"title": "Анкета сотрудника", "fields": ["ФИО", "должность"]},
    {"title": "Анкета безопасности", "fields": ["логин", "пароль"]},
]

for f in forms:
    if "пароль" in [x.lower() for x in f["fields"]]:
        print(f["title"], "-> Подозрительно (запрашивают пароль)")
    else:
        print(f["title"], "-> Норма")

Анкета сотрудника -> Норма
Анкета безопасности -> Подозрительно (запрашивают пароль)


👉 **Учебный момент:** пароли никогда не должны собираться через анкеты.

**Задание 18.** QR-фишинг (сканирование кода)

**Идея:** сгенерировать список QR-ссылок (текстом) и анализировать их.

In [ ]:
qrs = [
    "QR->https://www.company.com/promo",
    "QR->http://malicious.fakebank.ru/login"
]

for q in qrs:
    print(q, "->", "Подозрительно" if "http://" in q or "fakebank" in q else "Норма")

QR->https://www.company.com/promo -> Норма
QR->http://malicious.fakebank.ru/login -> Подозрительно


👉 **Учебный момент:** QR-коды тоже могут вести на фишинговые сайты.

**Задание 19.** Сценарий «подарок/выгода»

**Идея:** мошенники часто используют «бесплатные подарки» для сбора данных.

In [ ]:
offers = [
    "Поздравляем! Вы выиграли новый iPhone. Заполните форму для получения.",
    "У нас скидка 10% для сотрудников компании.",
]

for o in offers:
    if "выиграли" in o.lower() or "iphone" in o.lower():
        print(o, "-> Подозрительно (обещание подарка)")
    else:
        print(o, "-> Норма")

Поздравляем! Вы выиграли новый iPhone. Заполните форму для получения. -> Подозрительно (обещание подарка)
У нас скидка 10% для сотрудников компании. -> Норма


👉 **Учебный момент:** всё, что «слишком хорошо, чтобы быть правдой», = красный флаг.

**Задание 20.** Поддельные вакансии (HR-фишинг)

**Идея:** мошенники публикуют фальшивые вакансии, требуя «паспортные данные» или «предоплату».

In [ ]:
vacancies = [
    {"title": "Менеджер по продажам", "requirements": ["резюме", "паспорт"]},
    {"title": "Разработчик Python", "requirements": ["резюме", "портфолио"]},
]

for v in vacancies:
    if "паспорт" in [r.lower() for r in v["requirements"]]:
        print(v["title"], "-> Подозрительно (запрос паспорта)")
    else:
        print(v["title"], "-> Норма")

Менеджер по продажам -> Подозрительно (запрос паспорта)
Разработчик Python -> Норма


👉 **Учебный момент:** настоящие вакансии не требуют сканы документов на первом этапе.